In [17]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import requests
from PIL import Image

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-printed')




Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# load image from the IAM dataset
url = "test_value.jpg"
#image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
image = Image.open(url).convert("RGB")


pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)

In [24]:
generated_text

['42.']

Table OCR

In [ ]:
from transformers import TableTransformerModel, TableTransformerConfig

# Initializing a Table Transformer microsoft/table-transformer-detection style configuration
table_configuration = TableTransformerConfig()

# Initializing a model from the microsoft/table-transformer-detection style configuration
table_model = TableTransformerModel(table_configuration)

# Accessing the model configuration
table_configuration = table_model.config

In [1]:
from transformers import AutoImageProcessor, TableTransformerModel
from huggingface_hub import hf_hub_download
from PIL import Image

file_path = hf_hub_download(repo_id="nielsr/example-pdf", repo_type="dataset", filename="example_pdf.png")
image = Image.open(file_path).convert("RGB")

image_processor = AutoImageProcessor.from_pretrained("microsoft/table-transformer-detection")
table_model = TableTransformerModel.from_pretrained("microsoft/table-transformer-detection")

# prepare image for the model
inputs = image_processor(images=image, return_tensors="pt")

# forward pass
outputs = table_model(**inputs)

# the last hidden states are the final query embeddings of the Transformer decoder
# these are of shape (batch_size, num_queries, hidden_size)
last_hidden_states = outputs.last_hidden_state
list(last_hidden_states.shape)
outputs

/Users/I518302/opt/anaconda3/envs/linguistik/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


TableTransformerModelOutput(last_hidden_state=tensor([[[ 0.4487, -0.7559, -0.2180,  ...,  0.1938,  0.4595, -1.1296],
         [ 0.2736, -0.5557,  0.0671,  ...,  0.6335,  0.9419, -0.9986],
         [ 0.6326, -0.9232, -0.4263,  ...,  0.3511,  0.5909, -1.2945],
         ...,
         [ 0.5576, -0.8320, -0.2713,  ...,  0.3339,  0.6743, -1.2074],
         [ 0.5476, -0.9040, -0.3348,  ...,  0.2389,  0.5789, -1.2797],
         [ 0.4587, -0.8317,  0.0977,  ...,  0.9194,  1.2264, -0.9535]]],
       grad_fn=<NativeLayerNormBackward0>), past_key_values=None, decoder_hidden_states=None, decoder_attentions=None, cross_attentions=None, encoder_last_hidden_state=tensor([[[-0.0834,  0.0919,  0.0184,  ...,  0.6114,  0.0822, -0.2726],
         [-0.0707,  0.0313,  0.0669,  ...,  0.8984,  0.1130, -0.2027],
         [-0.0032,  0.0394,  0.0899,  ...,  0.8158,  0.1352, -0.3390],
         ...,
         [-0.0119,  0.1041, -0.0511,  ..., -0.3826,  0.1573, -0.6994],
         [ 0.0343,  0.1397, -0.0409,  ..., -0.

In [27]:
import re

from transformers import DonutProcessor, VisionEncoderDecoderModel
from datasets import load_dataset
import torch

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-rvlcdip")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-rvlcdip")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
# load document image
#dataset = load_dataset("hf-internal-testing/example-documents", split="test")
#image = dataset[1]["image"]

url = "test_value.jpg"
image = Image.open(url).convert("RGB")

# prepare decoder inputs
task_prompt = "<s_rvlcdip>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

pixel_values = processor(image, return_tensors="pt").pixel_values

outputs = model.generate(
    pixel_values.to(device),
    decoder_input_ids=decoder_input_ids.to(device),
    max_length=model.decoder.config.max_position_embeddings,
    early_stopping=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    use_cache=True,
    num_beams=1,
    bad_words_ids=[[processor.tokenizer.unk_token_id]],
    return_dict_in_generate=True,
)

sequence = processor.batch_decode(outputs.sequences)[0]
sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token
print(processor.token2json(sequence))

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


{'class': 'letter'}


In [30]:
dataset = load_dataset("hf-internal-testing/example-documents", split="test")
image = dataset[2]["image"]

task_prompt = "<s_cord-v2>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

pixel_values = processor(image, return_tensors="pt").pixel_values

outputs = model.generate(
    pixel_values.to(device),
    decoder_input_ids=decoder_input_ids.to(device),
    max_length=model.decoder.config.max_position_embeddings,
    early_stopping=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    use_cache=True,
    num_beams=1,
    bad_words_ids=[[processor.tokenizer.unk_token_id]],
    return_dict_in_generate=True,
)

sequence = processor.batch_decode(outputs.sequences)[0]
sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token
print(processor.token2json(sequence))

Found cached dataset parquet (/Users/I518302/.cache/huggingface/datasets/hf-internal-testing___parquet/hf-internal-testing--example-documents-2350012fcbf685c6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
Input length of decoder_input_ids is 8, but `max_length` is set to 8. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


{'text_sequence': '<presentation/>'}


In [31]:
import re

from transformers import DonutProcessor, VisionEncoderDecoderModel
from datasets import load_dataset
import torch

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
# load document image
dataset = load_dataset("hf-internal-testing/example-documents", split="test")
image = dataset[2]["image"]

# prepare decoder inputs
task_prompt = "<s_cord-v2>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

pixel_values = processor(image, return_tensors="pt").pixel_values

outputs = model.generate(
    pixel_values.to(device),
    decoder_input_ids=decoder_input_ids.to(device),
    max_length=model.decoder.config.max_position_embeddings,
    early_stopping=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    use_cache=True,
    num_beams=1,
    bad_words_ids=[[processor.tokenizer.unk_token_id]],
    return_dict_in_generate=True,
)

sequence = processor.batch_decode(outputs.sequences)[0]
sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token
print(processor.token2json(sequence))

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Found cached dataset parquet (/Users/I518302/.cache/huggingface/datasets/hf-internal-testing___parquet/hf-internal-testing--example-documents-2350012fcbf685c6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


{'menu': {'nm': 'CINNAMON SUGAR', 'unitprice': '17,000', 'cnt': '1 x', 'price': '17,000'}, 'sub_total': {'subtotal_price': '17,000'}, 'total': {'total_price': '17,000', 'cashprice': '20,000', 'changeprice': '3,000'}}


In [34]:
url = "test_table.jpg"
image = Image.open(url).convert("RGB")

In [35]:
# prepare decoder inputs
task_prompt = "<s_cord-v2>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

pixel_values = processor(image, return_tensors="pt").pixel_values

outputs = model.generate(
    pixel_values.to(device),
    decoder_input_ids=decoder_input_ids.to(device),
    max_length=model.decoder.config.max_position_embeddings,
    early_stopping=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    use_cache=True,
    num_beams=1,
    bad_words_ids=[[processor.tokenizer.unk_token_id]],
    return_dict_in_generate=True,
)

sequence = processor.batch_decode(outputs.sequences)[0]
sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token
print(processor.token2json(sequence))

{'menu': [{'nm': 'Pour 100 g', 'unitprice': 'portion - 200 g', 'cnt': '1', 'price': 'des AQR*'}, {'nm': 'energie 829 KJ-197 kcal 1658 KJ - 394 kcal', 'unitprice': '9,4g', 'cnt': '1', 'price': '20%'}, {'nm': 'Maiberes grasses', 'unitprice': '9,4g', 'cnt': '4,7g', 'price': '13%'}, {'nm': 'Dont acides gras satures', 'unitprice': '2,9g', 'cnt': '5,8g', 'price': '29%'}, {'nm': 'Glucides', 'cnt': '2,9', 'price': '20%'}, {'nm': 'Dont sucres', 'unitprice': '4,2g', 'cnt': '2,9', 'price': '5%'}, {'nm': 'Flores aimentaires', 'unitprice': '8,2g', 'cnt': '4,9', 'price': '40'}, {'nm': 'Sele', 'unitprice': '22g', 'cnt': '11g', 'price': '45 %'}], 'sub_total': {'subtotal_price': '3g', 'discount_price': '49%'}, 'total': {'total_price': '8 400 kJ/2000 kcal)'}}
